![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.svg)
# Pipeline 4 Notebook - AutoAI Notebook v1.22.7

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimal results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a>. 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for retrieving data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.10 and scikit-learn 1.1.1.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data](#read)<br>
&nbsp;&nbsp;[Train and test data split](#split)<br>
&nbsp;&nbsp;[Create pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
**[Store the model](#saving)**<br>
**[Summary and next steps](#summary_and_next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm-watsonx-ai,
 - autoai-libs,
 - scikit-learn


In [ ]:
!pip install ibm-watsonx-ai | tail -n 1
!pip install autoai-libs==1.17.2 | tail -n 1
!pip install scikit-learn==1.1.1 | tail -n 1
!pip install -U 'lale>=0.7,<0.8' | tail -n 1

Filter warnings for this notebook.

In [ ]:
import warnings

warnings.filterwarnings('ignore')

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [ ]:
from ibm_watsonx_ai.helpers import DataConnection
from ibm_watsonx_ai.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='785efa1c-d7d0-4957-976d-bf7ea5461cbf'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/0a469145-cd97-4648-88d9-6eef1be21a1c/wml_data/d23f07fe-20f7-4651-a368-3592d06338db/data/automl',
        model_location='auto_ml/0a469145-cd97-4648-88d9-6eef1be21a1c/wml_data/d23f07fe-20f7-4651-a368-3592d06338db/data/automl/model.zip',
        training_status='auto_ml/0a469145-cd97-4648-88d9-6eef1be21a1c/wml_data/d23f07fe-20f7-4651-a368-3592d06338db/training-status.json'
    )
)

The following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [ ]:
experiment_metadata = dict(
    prediction_type='regression',
    prediction_column='price',
    holdout_size=0.1,
    scoring='neg_root_mean_squared_error',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=2,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    deployment_url='https://eu-de.ml.cloud.ibm.com',
    project_id='3be8d731-bf36-41c3-8043-a7a5ea5965cb',
    drop_duplicates=True,
    include_batched_ensemble_estimators=[],
    feature_selector_mode='auto'
)

## Set `n_jobs` parameter to the number of available CPUs

In [ ]:
import os, ast
CPU_NUMBER = 1
if 'RUNTIME_HARDWARE_SPEC' in os.environ:
    CPU_NUMBER = int(ast.literal_eval(os.environ['RUNTIME_HARDWARE_SPEC'])['num_cpu'])

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

**Action**: Provide the IBM Cloud apikey, For details, see [documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey).

In [ ]:
api_key = 'PUT_YOUR_APIKEY_HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": experiment_metadata['deployment_url']
}

In [ ]:
from ibm_watsonx_ai import APIClient

wml_client = APIClient(wml_credentials)

if 'space_id' in experiment_metadata:
    wml_client.set.default_space(experiment_metadata['space_id'])
else:
    wml_client.set.default_project(experiment_metadata['project_id'])
    
training_data_references[0].set_client(wml_client)

<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

**Note**: If reading data results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`. 

It may be necessary to use methods for initial data pre-processing like: e.g. `DataFrame.dropna()`, `DataFrame.drop_duplicates()`, `DataFrame.sample()`, and outliers handling:

```
from autoai_libs.utils.outliers_mitigation import remove_outliers

df = remove_outliers(df, columns=[experiment_metadata['prediction_column']])
```


In [ ]:
X_train, X_test, y_train, y_test = training_data_references[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True, use_flight=True)

<a id="preview_model_to_python_code"></a>
## Create pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [ ]:
from autoai_libs.transformers.text_transformers import TextTransformer
from autoai_libs.transformers.date_time.date_time_transformer import (
    DateTransformer,
)
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TA1
import autoai_libs.utils.fc_methods
from autoai_libs.cognito.transforms.transform_utils import FS1
from autoai_libs.cognito.transforms.transform_utils import TAM
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

#### Pre-processing & Estimator.

In [ ]:
text_transformer_0 = TextTransformer(
    column_headers_list=[
        "date", "open", "high", "low", "NewTextFeature_0_word2vec(top_0)_0",
        "NewTextFeature_1_word2vec(top_0)_1",
        "NewTextFeature_2_word2vec(top_0)_2",
        "NewTextFeature_3_word2vec(top_0)_3",
        "NewTextFeature_4_word2vec(top_0)_4",
        "NewTextFeature_5_word2vec(top_0)_5",
        "NewTextFeature_6_word2vec(top_0)_6",
        "NewTextFeature_7_word2vec(top_0)_7",
        "NewTextFeature_8_word2vec(top_0)_8",
        "NewTextFeature_9_word2vec(top_0)_9",
        "NewTextFeature_10_word2vec(top_0)_10",
        "NewTextFeature_11_word2vec(top_0)_11",
        "NewTextFeature_12_word2vec(top_0)_12",
        "NewTextFeature_13_word2vec(top_0)_13",
        "NewTextFeature_14_word2vec(top_0)_14",
        "NewTextFeature_15_word2vec(top_0)_15",
        "NewTextFeature_16_word2vec(top_0)_16",
        "NewTextFeature_17_word2vec(top_0)_17",
        "NewTextFeature_18_word2vec(top_0)_18",
        "NewTextFeature_19_word2vec(top_0)_19",
        "NewTextFeature_20_word2vec(top_1)_20",
        "NewTextFeature_21_word2vec(top_1)_21",
        "NewTextFeature_22_word2vec(top_1)_22",
        "NewTextFeature_23_word2vec(top_1)_23",
        "NewTextFeature_24_word2vec(top_1)_24",
        "NewTextFeature_25_word2vec(top_1)_25",
        "NewTextFeature_26_word2vec(top_1)_26",
        "NewTextFeature_27_word2vec(top_1)_27",
        "NewTextFeature_28_word2vec(top_1)_28",
        "NewTextFeature_29_word2vec(top_1)_29",
        "NewTextFeature_30_word2vec(top_1)_30",
        "NewTextFeature_31_word2vec(top_1)_31",
        "NewTextFeature_32_word2vec(top_1)_32",
        "NewTextFeature_33_word2vec(top_1)_33",
        "NewTextFeature_34_word2vec(top_1)_34",
        "NewTextFeature_35_word2vec(top_1)_35",
        "NewTextFeature_36_word2vec(top_1)_36",
        "NewTextFeature_37_word2vec(top_1)_37",
        "NewTextFeature_38_word2vec(top_1)_38",
        "NewTextFeature_39_word2vec(top_1)_39",
        "NewTextFeature_40_word2vec(top_2)_40",
        "NewTextFeature_41_word2vec(top_2)_41",
        "NewTextFeature_42_word2vec(top_2)_42",
        "NewTextFeature_43_word2vec(top_2)_43",
        "NewTextFeature_44_word2vec(top_2)_44",
        "NewTextFeature_45_word2vec(top_2)_45",
        "NewTextFeature_46_word2vec(top_2)_46",
        "NewTextFeature_47_word2vec(top_2)_47",
        "NewTextFeature_48_word2vec(top_2)_48",
        "NewTextFeature_49_word2vec(top_2)_49",
        "NewTextFeature_50_word2vec(top_2)_50",
        "NewTextFeature_51_word2vec(top_2)_51",
        "NewTextFeature_52_word2vec(top_2)_52",
        "NewTextFeature_53_word2vec(top_2)_53",
        "NewTextFeature_54_word2vec(top_2)_54",
        "NewTextFeature_55_word2vec(top_2)_55",
        "NewTextFeature_56_word2vec(top_2)_56",
        "NewTextFeature_57_word2vec(top_2)_57",
        "NewTextFeature_58_word2vec(top_2)_58",
        "NewTextFeature_59_word2vec(top_2)_59",
        "NewTextFeature_60_word2vec(top_3)_60",
        "NewTextFeature_61_word2vec(top_3)_61",
        "NewTextFeature_62_word2vec(top_3)_62",
        "NewTextFeature_63_word2vec(top_3)_63",
        "NewTextFeature_64_word2vec(top_3)_64",
        "NewTextFeature_65_word2vec(top_3)_65",
        "NewTextFeature_66_word2vec(top_3)_66",
        "NewTextFeature_67_word2vec(top_3)_67",
        "NewTextFeature_68_word2vec(top_3)_68",
        "NewTextFeature_69_word2vec(top_3)_69",
        "NewTextFeature_70_word2vec(top_3)_70",
        "NewTextFeature_71_word2vec(top_3)_71",
        "NewTextFeature_72_word2vec(top_3)_72",
        "NewTextFeature_73_word2vec(top_3)_73",
        "NewTextFeature_74_word2vec(top_3)_74",
        "NewTextFeature_75_word2vec(top_3)_75",
        "NewTextFeature_76_word2vec(top_3)_76",
        "NewTextFeature_77_word2vec(top_3)_77",
        "NewTextFeature_78_word2vec(top_3)_78",
        "NewTextFeature_79_word2vec(top_3)_79",
        "NewTextFeature_80_word2vec(top_4)_80",
        "NewTextFeature_81_word2vec(top_4)_81",
        "NewTextFeature_82_word2vec(top_4)_82",
        "NewTextFeature_83_word2vec(top_4)_83",
        "NewTextFeature_84_word2vec(top_4)_84",
        "NewTextFeature_85_word2vec(top_4)_85",
        "NewTextFeature_86_word2vec(top_4)_86",
        "NewTextFeature_87_word2vec(top_4)_87",
        "NewTextFeature_88_word2vec(top_4)_88",
        "NewTextFeature_89_word2vec(top_4)_89",
        "NewTextFeature_90_word2vec(top_4)_90",
        "NewTextFeature_91_word2vec(top_4)_91",
        "NewTextFeature_92_word2vec(top_4)_92",
        "NewTextFeature_93_word2vec(top_4)_93",
        "NewTextFeature_94_word2vec(top_4)_94",
        "NewTextFeature_95_word2vec(top_4)_95",
        "NewTextFeature_96_word2vec(top_4)_96",
        "NewTextFeature_97_word2vec(top_4)_97",
        "NewTextFeature_98_word2vec(top_4)_98",
        "NewTextFeature_99_word2vec(top_4)_99",
        "NewTextFeature_100_word2vec(top_5)_100",
        "NewTextFeature_101_word2vec(top_5)_101",
        "NewTextFeature_102_word2vec(top_5)_102",
        "NewTextFeature_103_word2vec(top_5)_103",
        "NewTextFeature_104_word2vec(top_5)_104",
        "NewTextFeature_105_word2vec(top_5)_105",
        "NewTextFeature_106_word2vec(top_5)_106",
        "NewTextFeature_107_word2vec(top_5)_107",
        "NewTextFeature_108_word2vec(top_5)_108",
        "NewTextFeature_109_word2vec(top_5)_109",
        "NewTextFeature_110_word2vec(top_5)_110",
        "NewTextFeature_111_word2vec(top_5)_111",
        "NewTextFeature_112_word2vec(top_5)_112",
        "NewTextFeature_113_word2vec(top_5)_113",
        "NewTextFeature_114_word2vec(top_5)_114",
        "NewTextFeature_115_word2vec(top_5)_115",
        "NewTextFeature_116_word2vec(top_5)_116",
        "NewTextFeature_117_word2vec(top_5)_117",
        "NewTextFeature_118_word2vec(top_5)_118",
        "NewTextFeature_119_word2vec(top_5)_119",
        "NewTextFeature_120_word2vec(top_6)_120",
        "NewTextFeature_121_word2vec(top_6)_121",
        "NewTextFeature_122_word2vec(top_6)_122",
        "NewTextFeature_123_word2vec(top_6)_123",
        "NewTextFeature_124_word2vec(top_6)_124",
        "NewTextFeature_125_word2vec(top_6)_125",
        "NewTextFeature_126_word2vec(top_6)_126",
        "NewTextFeature_127_word2vec(top_6)_127",
        "NewTextFeature_128_word2vec(top_6)_128",
        "NewTextFeature_129_word2vec(top_6)_129",
        "NewTextFeature_130_word2vec(top_6)_130",
        "NewTextFeature_131_word2vec(top_6)_131",
        "NewTextFeature_132_word2vec(top_6)_132",
        "NewTextFeature_133_word2vec(top_6)_133",
        "NewTextFeature_134_word2vec(top_6)_134",
        "NewTextFeature_135_word2vec(top_6)_135",
        "NewTextFeature_136_word2vec(top_6)_136",
        "NewTextFeature_137_word2vec(top_6)_137",
        "NewTextFeature_138_word2vec(top_6)_138",
        "NewTextFeature_139_word2vec(top_6)_139",
        "NewTextFeature_140_word2vec(top_7)_140",
        "NewTextFeature_141_word2vec(top_7)_141",
        "NewTextFeature_142_word2vec(top_7)_142",
        "NewTextFeature_143_word2vec(top_7)_143",
        "NewTextFeature_144_word2vec(top_7)_144",
        "NewTextFeature_145_word2vec(top_7)_145",
        "NewTextFeature_146_word2vec(top_7)_146",
        "NewTextFeature_147_word2vec(top_7)_147",
        "NewTextFeature_148_word2vec(top_7)_148",
        "NewTextFeature_149_word2vec(top_7)_149",
        "NewTextFeature_150_word2vec(top_7)_150",
        "NewTextFeature_151_word2vec(top_7)_151",
        "NewTextFeature_152_word2vec(top_7)_152",
        "NewTextFeature_153_word2vec(top_7)_153",
        "NewTextFeature_154_word2vec(top_7)_154",
        "NewTextFeature_155_word2vec(top_7)_155",
        "NewTextFeature_156_word2vec(top_7)_156",
        "NewTextFeature_157_word2vec(top_7)_157",
        "NewTextFeature_158_word2vec(top_7)_158",
        "NewTextFeature_159_word2vec(top_7)_159",
        "NewTextFeature_160_word2vec(top_8)_160",
        "NewTextFeature_161_word2vec(top_8)_161",
        "NewTextFeature_162_word2vec(top_8)_162",
        "NewTextFeature_163_word2vec(top_8)_163",
        "NewTextFeature_164_word2vec(top_8)_164",
        "NewTextFeature_165_word2vec(top_8)_165",
        "NewTextFeature_166_word2vec(top_8)_166",
        "NewTextFeature_167_word2vec(top_8)_167",
        "NewTextFeature_168_word2vec(top_8)_168",
        "NewTextFeature_169_word2vec(top_8)_169",
        "NewTextFeature_170_word2vec(top_8)_170",
        "NewTextFeature_171_word2vec(top_8)_171",
        "NewTextFeature_172_word2vec(top_8)_172",
        "NewTextFeature_173_word2vec(top_8)_173",
        "NewTextFeature_174_word2vec(top_8)_174",
        "NewTextFeature_175_word2vec(top_8)_175",
        "NewTextFeature_176_word2vec(top_8)_176",
        "NewTextFeature_177_word2vec(top_8)_177",
        "NewTextFeature_178_word2vec(top_8)_178",
        "NewTextFeature_179_word2vec(top_8)_179",
    ],
    columns_to_be_deleted=[
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
    ],
    drop_columns=True,
    text_columns=[1, 2, 3, 4, 5, 6, 7, 8, 9],
    text_processing_options={"word2vec": {"output_dim": 20}},
)
date_transformer_0 = DateTransformer(
    column_headers_list=[
        "date", "open", "high", "low", "NewTextFeature_0_word2vec(top_0)_0",
        "NewTextFeature_1_word2vec(top_0)_1",
        "NewTextFeature_2_word2vec(top_0)_2",
        "NewTextFeature_3_word2vec(top_0)_3",
        "NewTextFeature_4_word2vec(top_0)_4",
        "NewTextFeature_5_word2vec(top_0)_5",
        "NewTextFeature_6_word2vec(top_0)_6",
        "NewTextFeature_7_word2vec(top_0)_7",
        "NewTextFeature_8_word2vec(top_0)_8",
        "NewTextFeature_9_word2vec(top_0)_9",
        "NewTextFeature_10_word2vec(top_0)_10",
        "NewTextFeature_11_word2vec(top_0)_11",
        "NewTextFeature_12_word2vec(top_0)_12",
        "NewTextFeature_13_word2vec(top_0)_13",
        "NewTextFeature_14_word2vec(top_0)_14",
        "NewTextFeature_15_word2vec(top_0)_15",
        "NewTextFeature_16_word2vec(top_0)_16",
        "NewTextFeature_17_word2vec(top_0)_17",
        "NewTextFeature_18_word2vec(top_0)_18",
        "NewTextFeature_19_word2vec(top_0)_19",
        "NewTextFeature_20_word2vec(top_1)_20",
        "NewTextFeature_21_word2vec(top_1)_21",
        "NewTextFeature_22_word2vec(top_1)_22",
        "NewTextFeature_23_word2vec(top_1)_23",
        "NewTextFeature_24_word2vec(top_1)_24",
        "NewTextFeature_25_word2vec(top_1)_25",
        "NewTextFeature_26_word2vec(top_1)_26",
        "NewTextFeature_27_word2vec(top_1)_27",
        "NewTextFeature_28_word2vec(top_1)_28",
        "NewTextFeature_29_word2vec(top_1)_29",
        "NewTextFeature_30_word2vec(top_1)_30",
        "NewTextFeature_31_word2vec(top_1)_31",
        "NewTextFeature_32_word2vec(top_1)_32",
        "NewTextFeature_33_word2vec(top_1)_33",
        "NewTextFeature_34_word2vec(top_1)_34",
        "NewTextFeature_35_word2vec(top_1)_35",
        "NewTextFeature_36_word2vec(top_1)_36",
        "NewTextFeature_37_word2vec(top_1)_37",
        "NewTextFeature_38_word2vec(top_1)_38",
        "NewTextFeature_39_word2vec(top_1)_39",
        "NewTextFeature_40_word2vec(top_2)_40",
        "NewTextFeature_41_word2vec(top_2)_41",
        "NewTextFeature_42_word2vec(top_2)_42",
        "NewTextFeature_43_word2vec(top_2)_43",
        "NewTextFeature_44_word2vec(top_2)_44",
        "NewTextFeature_45_word2vec(top_2)_45",
        "NewTextFeature_46_word2vec(top_2)_46",
        "NewTextFeature_47_word2vec(top_2)_47",
        "NewTextFeature_48_word2vec(top_2)_48",
        "NewTextFeature_49_word2vec(top_2)_49",
        "NewTextFeature_50_word2vec(top_2)_50",
        "NewTextFeature_51_word2vec(top_2)_51",
        "NewTextFeature_52_word2vec(top_2)_52",
        "NewTextFeature_53_word2vec(top_2)_53",
        "NewTextFeature_54_word2vec(top_2)_54",
        "NewTextFeature_55_word2vec(top_2)_55",
        "NewTextFeature_56_word2vec(top_2)_56",
        "NewTextFeature_57_word2vec(top_2)_57",
        "NewTextFeature_58_word2vec(top_2)_58",
        "NewTextFeature_59_word2vec(top_2)_59",
        "NewTextFeature_60_word2vec(top_3)_60",
        "NewTextFeature_61_word2vec(top_3)_61",
        "NewTextFeature_62_word2vec(top_3)_62",
        "NewTextFeature_63_word2vec(top_3)_63",
        "NewTextFeature_64_word2vec(top_3)_64",
        "NewTextFeature_65_word2vec(top_3)_65",
        "NewTextFeature_66_word2vec(top_3)_66",
        "NewTextFeature_67_word2vec(top_3)_67",
        "NewTextFeature_68_word2vec(top_3)_68",
        "NewTextFeature_69_word2vec(top_3)_69",
        "NewTextFeature_70_word2vec(top_3)_70",
        "NewTextFeature_71_word2vec(top_3)_71",
        "NewTextFeature_72_word2vec(top_3)_72",
        "NewTextFeature_73_word2vec(top_3)_73",
        "NewTextFeature_74_word2vec(top_3)_74",
        "NewTextFeature_75_word2vec(top_3)_75",
        "NewTextFeature_76_word2vec(top_3)_76",
        "NewTextFeature_77_word2vec(top_3)_77",
        "NewTextFeature_78_word2vec(top_3)_78",
        "NewTextFeature_79_word2vec(top_3)_79",
        "NewTextFeature_80_word2vec(top_4)_80",
        "NewTextFeature_81_word2vec(top_4)_81",
        "NewTextFeature_82_word2vec(top_4)_82",
        "NewTextFeature_83_word2vec(top_4)_83",
        "NewTextFeature_84_word2vec(top_4)_84",
        "NewTextFeature_85_word2vec(top_4)_85",
        "NewTextFeature_86_word2vec(top_4)_86",
        "NewTextFeature_87_word2vec(top_4)_87",
        "NewTextFeature_88_word2vec(top_4)_88",
        "NewTextFeature_89_word2vec(top_4)_89",
        "NewTextFeature_90_word2vec(top_4)_90",
        "NewTextFeature_91_word2vec(top_4)_91",
        "NewTextFeature_92_word2vec(top_4)_92",
        "NewTextFeature_93_word2vec(top_4)_93",
        "NewTextFeature_94_word2vec(top_4)_94",
        "NewTextFeature_95_word2vec(top_4)_95",
        "NewTextFeature_96_word2vec(top_4)_96",
        "NewTextFeature_97_word2vec(top_4)_97",
        "NewTextFeature_98_word2vec(top_4)_98",
        "NewTextFeature_99_word2vec(top_4)_99",
        "NewTextFeature_100_word2vec(top_5)_100",
        "NewTextFeature_101_word2vec(top_5)_101",
        "NewTextFeature_102_word2vec(top_5)_102",
        "NewTextFeature_103_word2vec(top_5)_103",
        "NewTextFeature_104_word2vec(top_5)_104",
        "NewTextFeature_105_word2vec(top_5)_105",
        "NewTextFeature_106_word2vec(top_5)_106",
        "NewTextFeature_107_word2vec(top_5)_107",
        "NewTextFeature_108_word2vec(top_5)_108",
        "NewTextFeature_109_word2vec(top_5)_109",
        "NewTextFeature_110_word2vec(top_5)_110",
        "NewTextFeature_111_word2vec(top_5)_111",
        "NewTextFeature_112_word2vec(top_5)_112",
        "NewTextFeature_113_word2vec(top_5)_113",
        "NewTextFeature_114_word2vec(top_5)_114",
        "NewTextFeature_115_word2vec(top_5)_115",
        "NewTextFeature_116_word2vec(top_5)_116",
        "NewTextFeature_117_word2vec(top_5)_117",
        "NewTextFeature_118_word2vec(top_5)_118",
        "NewTextFeature_119_word2vec(top_5)_119",
        "NewTextFeature_120_word2vec(top_6)_120",
        "NewTextFeature_121_word2vec(top_6)_121",
        "NewTextFeature_122_word2vec(top_6)_122",
        "NewTextFeature_123_word2vec(top_6)_123",
        "NewTextFeature_124_word2vec(top_6)_124",
        "NewTextFeature_125_word2vec(top_6)_125",
        "NewTextFeature_126_word2vec(top_6)_126",
        "NewTextFeature_127_word2vec(top_6)_127",
        "NewTextFeature_128_word2vec(top_6)_128",
        "NewTextFeature_129_word2vec(top_6)_129",
        "NewTextFeature_130_word2vec(top_6)_130",
        "NewTextFeature_131_word2vec(top_6)_131",
        "NewTextFeature_132_word2vec(top_6)_132",
        "NewTextFeature_133_word2vec(top_6)_133",
        "NewTextFeature_134_word2vec(top_6)_134",
        "NewTextFeature_135_word2vec(top_6)_135",
        "NewTextFeature_136_word2vec(top_6)_136",
        "NewTextFeature_137_word2vec(top_6)_137",
        "NewTextFeature_138_word2vec(top_6)_138",
        "NewTextFeature_139_word2vec(top_6)_139",
        "NewTextFeature_140_word2vec(top_7)_140",
        "NewTextFeature_141_word2vec(top_7)_141",
        "NewTextFeature_142_word2vec(top_7)_142",
        "NewTextFeature_143_word2vec(top_7)_143",
        "NewTextFeature_144_word2vec(top_7)_144",
        "NewTextFeature_145_word2vec(top_7)_145",
        "NewTextFeature_146_word2vec(top_7)_146",
        "NewTextFeature_147_word2vec(top_7)_147",
        "NewTextFeature_148_word2vec(top_7)_148",
        "NewTextFeature_149_word2vec(top_7)_149",
        "NewTextFeature_150_word2vec(top_7)_150",
        "NewTextFeature_151_word2vec(top_7)_151",
        "NewTextFeature_152_word2vec(top_7)_152",
        "NewTextFeature_153_word2vec(top_7)_153",
        "NewTextFeature_154_word2vec(top_7)_154",
        "NewTextFeature_155_word2vec(top_7)_155",
        "NewTextFeature_156_word2vec(top_7)_156",
        "NewTextFeature_157_word2vec(top_7)_157",
        "NewTextFeature_158_word2vec(top_7)_158",
        "NewTextFeature_159_word2vec(top_7)_159",
        "NewTextFeature_160_word2vec(top_8)_160",
        "NewTextFeature_161_word2vec(top_8)_161",
        "NewTextFeature_162_word2vec(top_8)_162",
        "NewTextFeature_163_word2vec(top_8)_163",
        "NewTextFeature_164_word2vec(top_8)_164",
        "NewTextFeature_165_word2vec(top_8)_165",
        "NewTextFeature_166_word2vec(top_8)_166",
        "NewTextFeature_167_word2vec(top_8)_167",
        "NewTextFeature_168_word2vec(top_8)_168",
        "NewTextFeature_169_word2vec(top_8)_169",
        "NewTextFeature_170_word2vec(top_8)_170",
        "NewTextFeature_171_word2vec(top_8)_171",
        "NewTextFeature_172_word2vec(top_8)_172",
        "NewTextFeature_173_word2vec(top_8)_173",
        "NewTextFeature_174_word2vec(top_8)_174",
        "NewTextFeature_175_word2vec(top_8)_175",
        "NewTextFeature_176_word2vec(top_8)_176",
        "NewTextFeature_177_word2vec(top_8)_177",
        "NewTextFeature_178_word2vec(top_8)_178",
        "NewTextFeature_179_word2vec(top_8)_179",
    ],
    missing_values_reference_list=["?", "", "-", float("nan")],
    options=["all"],
)
numpy_column_selector_0 = NumpyColumnSelector(
    columns=[184, 185, 186, 188, 189]
)
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=["int_num", "int_num", "int_num", "int_num", "int_num"],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[[], [], [], [], []],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    filling_values=float("nan"), missing_values=[]
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    missing_values=float("nan"),
    sklearn_version_family="1",
    strategy="most_frequent",
)
cat_encoder = CatEncoder(
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="1",
    encoding="ordinal",
    categories="auto",
)
pipeline_0 = make_pipeline(
    text_transformer_0,
    date_transformer_0,
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
text_transformer_1 = TextTransformer(
    column_headers_list=[
        "date", "open", "high", "low", "NewTextFeature_0_word2vec(top_0)_0",
        "NewTextFeature_1_word2vec(top_0)_1",
        "NewTextFeature_2_word2vec(top_0)_2",
        "NewTextFeature_3_word2vec(top_0)_3",
        "NewTextFeature_4_word2vec(top_0)_4",
        "NewTextFeature_5_word2vec(top_0)_5",
        "NewTextFeature_6_word2vec(top_0)_6",
        "NewTextFeature_7_word2vec(top_0)_7",
        "NewTextFeature_8_word2vec(top_0)_8",
        "NewTextFeature_9_word2vec(top_0)_9",
        "NewTextFeature_10_word2vec(top_0)_10",
        "NewTextFeature_11_word2vec(top_0)_11",
        "NewTextFeature_12_word2vec(top_0)_12",
        "NewTextFeature_13_word2vec(top_0)_13",
        "NewTextFeature_14_word2vec(top_0)_14",
        "NewTextFeature_15_word2vec(top_0)_15",
        "NewTextFeature_16_word2vec(top_0)_16",
        "NewTextFeature_17_word2vec(top_0)_17",
        "NewTextFeature_18_word2vec(top_0)_18",
        "NewTextFeature_19_word2vec(top_0)_19",
        "NewTextFeature_20_word2vec(top_1)_20",
        "NewTextFeature_21_word2vec(top_1)_21",
        "NewTextFeature_22_word2vec(top_1)_22",
        "NewTextFeature_23_word2vec(top_1)_23",
        "NewTextFeature_24_word2vec(top_1)_24",
        "NewTextFeature_25_word2vec(top_1)_25",
        "NewTextFeature_26_word2vec(top_1)_26",
        "NewTextFeature_27_word2vec(top_1)_27",
        "NewTextFeature_28_word2vec(top_1)_28",
        "NewTextFeature_29_word2vec(top_1)_29",
        "NewTextFeature_30_word2vec(top_1)_30",
        "NewTextFeature_31_word2vec(top_1)_31",
        "NewTextFeature_32_word2vec(top_1)_32",
        "NewTextFeature_33_word2vec(top_1)_33",
        "NewTextFeature_34_word2vec(top_1)_34",
        "NewTextFeature_35_word2vec(top_1)_35",
        "NewTextFeature_36_word2vec(top_1)_36",
        "NewTextFeature_37_word2vec(top_1)_37",
        "NewTextFeature_38_word2vec(top_1)_38",
        "NewTextFeature_39_word2vec(top_1)_39",
        "NewTextFeature_40_word2vec(top_2)_40",
        "NewTextFeature_41_word2vec(top_2)_41",
        "NewTextFeature_42_word2vec(top_2)_42",
        "NewTextFeature_43_word2vec(top_2)_43",
        "NewTextFeature_44_word2vec(top_2)_44",
        "NewTextFeature_45_word2vec(top_2)_45",
        "NewTextFeature_46_word2vec(top_2)_46",
        "NewTextFeature_47_word2vec(top_2)_47",
        "NewTextFeature_48_word2vec(top_2)_48",
        "NewTextFeature_49_word2vec(top_2)_49",
        "NewTextFeature_50_word2vec(top_2)_50",
        "NewTextFeature_51_word2vec(top_2)_51",
        "NewTextFeature_52_word2vec(top_2)_52",
        "NewTextFeature_53_word2vec(top_2)_53",
        "NewTextFeature_54_word2vec(top_2)_54",
        "NewTextFeature_55_word2vec(top_2)_55",
        "NewTextFeature_56_word2vec(top_2)_56",
        "NewTextFeature_57_word2vec(top_2)_57",
        "NewTextFeature_58_word2vec(top_2)_58",
        "NewTextFeature_59_word2vec(top_2)_59",
        "NewTextFeature_60_word2vec(top_3)_60",
        "NewTextFeature_61_word2vec(top_3)_61",
        "NewTextFeature_62_word2vec(top_3)_62",
        "NewTextFeature_63_word2vec(top_3)_63",
        "NewTextFeature_64_word2vec(top_3)_64",
        "NewTextFeature_65_word2vec(top_3)_65",
        "NewTextFeature_66_word2vec(top_3)_66",
        "NewTextFeature_67_word2vec(top_3)_67",
        "NewTextFeature_68_word2vec(top_3)_68",
        "NewTextFeature_69_word2vec(top_3)_69",
        "NewTextFeature_70_word2vec(top_3)_70",
        "NewTextFeature_71_word2vec(top_3)_71",
        "NewTextFeature_72_word2vec(top_3)_72",
        "NewTextFeature_73_word2vec(top_3)_73",
        "NewTextFeature_74_word2vec(top_3)_74",
        "NewTextFeature_75_word2vec(top_3)_75",
        "NewTextFeature_76_word2vec(top_3)_76",
        "NewTextFeature_77_word2vec(top_3)_77",
        "NewTextFeature_78_word2vec(top_3)_78",
        "NewTextFeature_79_word2vec(top_3)_79",
        "NewTextFeature_80_word2vec(top_4)_80",
        "NewTextFeature_81_word2vec(top_4)_81",
        "NewTextFeature_82_word2vec(top_4)_82",
        "NewTextFeature_83_word2vec(top_4)_83",
        "NewTextFeature_84_word2vec(top_4)_84",
        "NewTextFeature_85_word2vec(top_4)_85",
        "NewTextFeature_86_word2vec(top_4)_86",
        "NewTextFeature_87_word2vec(top_4)_87",
        "NewTextFeature_88_word2vec(top_4)_88",
        "NewTextFeature_89_word2vec(top_4)_89",
        "NewTextFeature_90_word2vec(top_4)_90",
        "NewTextFeature_91_word2vec(top_4)_91",
        "NewTextFeature_92_word2vec(top_4)_92",
        "NewTextFeature_93_word2vec(top_4)_93",
        "NewTextFeature_94_word2vec(top_4)_94",
        "NewTextFeature_95_word2vec(top_4)_95",
        "NewTextFeature_96_word2vec(top_4)_96",
        "NewTextFeature_97_word2vec(top_4)_97",
        "NewTextFeature_98_word2vec(top_4)_98",
        "NewTextFeature_99_word2vec(top_4)_99",
        "NewTextFeature_100_word2vec(top_5)_100",
        "NewTextFeature_101_word2vec(top_5)_101",
        "NewTextFeature_102_word2vec(top_5)_102",
        "NewTextFeature_103_word2vec(top_5)_103",
        "NewTextFeature_104_word2vec(top_5)_104",
        "NewTextFeature_105_word2vec(top_5)_105",
        "NewTextFeature_106_word2vec(top_5)_106",
        "NewTextFeature_107_word2vec(top_5)_107",
        "NewTextFeature_108_word2vec(top_5)_108",
        "NewTextFeature_109_word2vec(top_5)_109",
        "NewTextFeature_110_word2vec(top_5)_110",
        "NewTextFeature_111_word2vec(top_5)_111",
        "NewTextFeature_112_word2vec(top_5)_112",
        "NewTextFeature_113_word2vec(top_5)_113",
        "NewTextFeature_114_word2vec(top_5)_114",
        "NewTextFeature_115_word2vec(top_5)_115",
        "NewTextFeature_116_word2vec(top_5)_116",
        "NewTextFeature_117_word2vec(top_5)_117",
        "NewTextFeature_118_word2vec(top_5)_118",
        "NewTextFeature_119_word2vec(top_5)_119",
        "NewTextFeature_120_word2vec(top_6)_120",
        "NewTextFeature_121_word2vec(top_6)_121",
        "NewTextFeature_122_word2vec(top_6)_122",
        "NewTextFeature_123_word2vec(top_6)_123",
        "NewTextFeature_124_word2vec(top_6)_124",
        "NewTextFeature_125_word2vec(top_6)_125",
        "NewTextFeature_126_word2vec(top_6)_126",
        "NewTextFeature_127_word2vec(top_6)_127",
        "NewTextFeature_128_word2vec(top_6)_128",
        "NewTextFeature_129_word2vec(top_6)_129",
        "NewTextFeature_130_word2vec(top_6)_130",
        "NewTextFeature_131_word2vec(top_6)_131",
        "NewTextFeature_132_word2vec(top_6)_132",
        "NewTextFeature_133_word2vec(top_6)_133",
        "NewTextFeature_134_word2vec(top_6)_134",
        "NewTextFeature_135_word2vec(top_6)_135",
        "NewTextFeature_136_word2vec(top_6)_136",
        "NewTextFeature_137_word2vec(top_6)_137",
        "NewTextFeature_138_word2vec(top_6)_138",
        "NewTextFeature_139_word2vec(top_6)_139",
        "NewTextFeature_140_word2vec(top_7)_140",
        "NewTextFeature_141_word2vec(top_7)_141",
        "NewTextFeature_142_word2vec(top_7)_142",
        "NewTextFeature_143_word2vec(top_7)_143",
        "NewTextFeature_144_word2vec(top_7)_144",
        "NewTextFeature_145_word2vec(top_7)_145",
        "NewTextFeature_146_word2vec(top_7)_146",
        "NewTextFeature_147_word2vec(top_7)_147",
        "NewTextFeature_148_word2vec(top_7)_148",
        "NewTextFeature_149_word2vec(top_7)_149",
        "NewTextFeature_150_word2vec(top_7)_150",
        "NewTextFeature_151_word2vec(top_7)_151",
        "NewTextFeature_152_word2vec(top_7)_152",
        "NewTextFeature_153_word2vec(top_7)_153",
        "NewTextFeature_154_word2vec(top_7)_154",
        "NewTextFeature_155_word2vec(top_7)_155",
        "NewTextFeature_156_word2vec(top_7)_156",
        "NewTextFeature_157_word2vec(top_7)_157",
        "NewTextFeature_158_word2vec(top_7)_158",
        "NewTextFeature_159_word2vec(top_7)_159",
        "NewTextFeature_160_word2vec(top_8)_160",
        "NewTextFeature_161_word2vec(top_8)_161",
        "NewTextFeature_162_word2vec(top_8)_162",
        "NewTextFeature_163_word2vec(top_8)_163",
        "NewTextFeature_164_word2vec(top_8)_164",
        "NewTextFeature_165_word2vec(top_8)_165",
        "NewTextFeature_166_word2vec(top_8)_166",
        "NewTextFeature_167_word2vec(top_8)_167",
        "NewTextFeature_168_word2vec(top_8)_168",
        "NewTextFeature_169_word2vec(top_8)_169",
        "NewTextFeature_170_word2vec(top_8)_170",
        "NewTextFeature_171_word2vec(top_8)_171",
        "NewTextFeature_172_word2vec(top_8)_172",
        "NewTextFeature_173_word2vec(top_8)_173",
        "NewTextFeature_174_word2vec(top_8)_174",
        "NewTextFeature_175_word2vec(top_8)_175",
        "NewTextFeature_176_word2vec(top_8)_176",
        "NewTextFeature_177_word2vec(top_8)_177",
        "NewTextFeature_178_word2vec(top_8)_178",
        "NewTextFeature_179_word2vec(top_8)_179",
    ],
    columns_to_be_deleted=[
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
    ],
    drop_columns=True,
    text_columns=[1, 2, 3, 4, 5, 6, 7, 8, 9],
    text_processing_options={"word2vec": {"output_dim": 20}},
)
date_transformer_1 = DateTransformer(
    column_headers_list=[
        "date", "open", "high", "low", "NewTextFeature_0_word2vec(top_0)_0",
        "NewTextFeature_1_word2vec(top_0)_1",
        "NewTextFeature_2_word2vec(top_0)_2",
        "NewTextFeature_3_word2vec(top_0)_3",
        "NewTextFeature_4_word2vec(top_0)_4",
        "NewTextFeature_5_word2vec(top_0)_5",
        "NewTextFeature_6_word2vec(top_0)_6",
        "NewTextFeature_7_word2vec(top_0)_7",
        "NewTextFeature_8_word2vec(top_0)_8",
        "NewTextFeature_9_word2vec(top_0)_9",
        "NewTextFeature_10_word2vec(top_0)_10",
        "NewTextFeature_11_word2vec(top_0)_11",
        "NewTextFeature_12_word2vec(top_0)_12",
        "NewTextFeature_13_word2vec(top_0)_13",
        "NewTextFeature_14_word2vec(top_0)_14",
        "NewTextFeature_15_word2vec(top_0)_15",
        "NewTextFeature_16_word2vec(top_0)_16",
        "NewTextFeature_17_word2vec(top_0)_17",
        "NewTextFeature_18_word2vec(top_0)_18",
        "NewTextFeature_19_word2vec(top_0)_19",
        "NewTextFeature_20_word2vec(top_1)_20",
        "NewTextFeature_21_word2vec(top_1)_21",
        "NewTextFeature_22_word2vec(top_1)_22",
        "NewTextFeature_23_word2vec(top_1)_23",
        "NewTextFeature_24_word2vec(top_1)_24",
        "NewTextFeature_25_word2vec(top_1)_25",
        "NewTextFeature_26_word2vec(top_1)_26",
        "NewTextFeature_27_word2vec(top_1)_27",
        "NewTextFeature_28_word2vec(top_1)_28",
        "NewTextFeature_29_word2vec(top_1)_29",
        "NewTextFeature_30_word2vec(top_1)_30",
        "NewTextFeature_31_word2vec(top_1)_31",
        "NewTextFeature_32_word2vec(top_1)_32",
        "NewTextFeature_33_word2vec(top_1)_33",
        "NewTextFeature_34_word2vec(top_1)_34",
        "NewTextFeature_35_word2vec(top_1)_35",
        "NewTextFeature_36_word2vec(top_1)_36",
        "NewTextFeature_37_word2vec(top_1)_37",
        "NewTextFeature_38_word2vec(top_1)_38",
        "NewTextFeature_39_word2vec(top_1)_39",
        "NewTextFeature_40_word2vec(top_2)_40",
        "NewTextFeature_41_word2vec(top_2)_41",
        "NewTextFeature_42_word2vec(top_2)_42",
        "NewTextFeature_43_word2vec(top_2)_43",
        "NewTextFeature_44_word2vec(top_2)_44",
        "NewTextFeature_45_word2vec(top_2)_45",
        "NewTextFeature_46_word2vec(top_2)_46",
        "NewTextFeature_47_word2vec(top_2)_47",
        "NewTextFeature_48_word2vec(top_2)_48",
        "NewTextFeature_49_word2vec(top_2)_49",
        "NewTextFeature_50_word2vec(top_2)_50",
        "NewTextFeature_51_word2vec(top_2)_51",
        "NewTextFeature_52_word2vec(top_2)_52",
        "NewTextFeature_53_word2vec(top_2)_53",
        "NewTextFeature_54_word2vec(top_2)_54",
        "NewTextFeature_55_word2vec(top_2)_55",
        "NewTextFeature_56_word2vec(top_2)_56",
        "NewTextFeature_57_word2vec(top_2)_57",
        "NewTextFeature_58_word2vec(top_2)_58",
        "NewTextFeature_59_word2vec(top_2)_59",
        "NewTextFeature_60_word2vec(top_3)_60",
        "NewTextFeature_61_word2vec(top_3)_61",
        "NewTextFeature_62_word2vec(top_3)_62",
        "NewTextFeature_63_word2vec(top_3)_63",
        "NewTextFeature_64_word2vec(top_3)_64",
        "NewTextFeature_65_word2vec(top_3)_65",
        "NewTextFeature_66_word2vec(top_3)_66",
        "NewTextFeature_67_word2vec(top_3)_67",
        "NewTextFeature_68_word2vec(top_3)_68",
        "NewTextFeature_69_word2vec(top_3)_69",
        "NewTextFeature_70_word2vec(top_3)_70",
        "NewTextFeature_71_word2vec(top_3)_71",
        "NewTextFeature_72_word2vec(top_3)_72",
        "NewTextFeature_73_word2vec(top_3)_73",
        "NewTextFeature_74_word2vec(top_3)_74",
        "NewTextFeature_75_word2vec(top_3)_75",
        "NewTextFeature_76_word2vec(top_3)_76",
        "NewTextFeature_77_word2vec(top_3)_77",
        "NewTextFeature_78_word2vec(top_3)_78",
        "NewTextFeature_79_word2vec(top_3)_79",
        "NewTextFeature_80_word2vec(top_4)_80",
        "NewTextFeature_81_word2vec(top_4)_81",
        "NewTextFeature_82_word2vec(top_4)_82",
        "NewTextFeature_83_word2vec(top_4)_83",
        "NewTextFeature_84_word2vec(top_4)_84",
        "NewTextFeature_85_word2vec(top_4)_85",
        "NewTextFeature_86_word2vec(top_4)_86",
        "NewTextFeature_87_word2vec(top_4)_87",
        "NewTextFeature_88_word2vec(top_4)_88",
        "NewTextFeature_89_word2vec(top_4)_89",
        "NewTextFeature_90_word2vec(top_4)_90",
        "NewTextFeature_91_word2vec(top_4)_91",
        "NewTextFeature_92_word2vec(top_4)_92",
        "NewTextFeature_93_word2vec(top_4)_93",
        "NewTextFeature_94_word2vec(top_4)_94",
        "NewTextFeature_95_word2vec(top_4)_95",
        "NewTextFeature_96_word2vec(top_4)_96",
        "NewTextFeature_97_word2vec(top_4)_97",
        "NewTextFeature_98_word2vec(top_4)_98",
        "NewTextFeature_99_word2vec(top_4)_99",
        "NewTextFeature_100_word2vec(top_5)_100",
        "NewTextFeature_101_word2vec(top_5)_101",
        "NewTextFeature_102_word2vec(top_5)_102",
        "NewTextFeature_103_word2vec(top_5)_103",
        "NewTextFeature_104_word2vec(top_5)_104",
        "NewTextFeature_105_word2vec(top_5)_105",
        "NewTextFeature_106_word2vec(top_5)_106",
        "NewTextFeature_107_word2vec(top_5)_107",
        "NewTextFeature_108_word2vec(top_5)_108",
        "NewTextFeature_109_word2vec(top_5)_109",
        "NewTextFeature_110_word2vec(top_5)_110",
        "NewTextFeature_111_word2vec(top_5)_111",
        "NewTextFeature_112_word2vec(top_5)_112",
        "NewTextFeature_113_word2vec(top_5)_113",
        "NewTextFeature_114_word2vec(top_5)_114",
        "NewTextFeature_115_word2vec(top_5)_115",
        "NewTextFeature_116_word2vec(top_5)_116",
        "NewTextFeature_117_word2vec(top_5)_117",
        "NewTextFeature_118_word2vec(top_5)_118",
        "NewTextFeature_119_word2vec(top_5)_119",
        "NewTextFeature_120_word2vec(top_6)_120",
        "NewTextFeature_121_word2vec(top_6)_121",
        "NewTextFeature_122_word2vec(top_6)_122",
        "NewTextFeature_123_word2vec(top_6)_123",
        "NewTextFeature_124_word2vec(top_6)_124",
        "NewTextFeature_125_word2vec(top_6)_125",
        "NewTextFeature_126_word2vec(top_6)_126",
        "NewTextFeature_127_word2vec(top_6)_127",
        "NewTextFeature_128_word2vec(top_6)_128",
        "NewTextFeature_129_word2vec(top_6)_129",
        "NewTextFeature_130_word2vec(top_6)_130",
        "NewTextFeature_131_word2vec(top_6)_131",
        "NewTextFeature_132_word2vec(top_6)_132",
        "NewTextFeature_133_word2vec(top_6)_133",
        "NewTextFeature_134_word2vec(top_6)_134",
        "NewTextFeature_135_word2vec(top_6)_135",
        "NewTextFeature_136_word2vec(top_6)_136",
        "NewTextFeature_137_word2vec(top_6)_137",
        "NewTextFeature_138_word2vec(top_6)_138",
        "NewTextFeature_139_word2vec(top_6)_139",
        "NewTextFeature_140_word2vec(top_7)_140",
        "NewTextFeature_141_word2vec(top_7)_141",
        "NewTextFeature_142_word2vec(top_7)_142",
        "NewTextFeature_143_word2vec(top_7)_143",
        "NewTextFeature_144_word2vec(top_7)_144",
        "NewTextFeature_145_word2vec(top_7)_145",
        "NewTextFeature_146_word2vec(top_7)_146",
        "NewTextFeature_147_word2vec(top_7)_147",
        "NewTextFeature_148_word2vec(top_7)_148",
        "NewTextFeature_149_word2vec(top_7)_149",
        "NewTextFeature_150_word2vec(top_7)_150",
        "NewTextFeature_151_word2vec(top_7)_151",
        "NewTextFeature_152_word2vec(top_7)_152",
        "NewTextFeature_153_word2vec(top_7)_153",
        "NewTextFeature_154_word2vec(top_7)_154",
        "NewTextFeature_155_word2vec(top_7)_155",
        "NewTextFeature_156_word2vec(top_7)_156",
        "NewTextFeature_157_word2vec(top_7)_157",
        "NewTextFeature_158_word2vec(top_7)_158",
        "NewTextFeature_159_word2vec(top_7)_159",
        "NewTextFeature_160_word2vec(top_8)_160",
        "NewTextFeature_161_word2vec(top_8)_161",
        "NewTextFeature_162_word2vec(top_8)_162",
        "NewTextFeature_163_word2vec(top_8)_163",
        "NewTextFeature_164_word2vec(top_8)_164",
        "NewTextFeature_165_word2vec(top_8)_165",
        "NewTextFeature_166_word2vec(top_8)_166",
        "NewTextFeature_167_word2vec(top_8)_167",
        "NewTextFeature_168_word2vec(top_8)_168",
        "NewTextFeature_169_word2vec(top_8)_169",
        "NewTextFeature_170_word2vec(top_8)_170",
        "NewTextFeature_171_word2vec(top_8)_171",
        "NewTextFeature_172_word2vec(top_8)_172",
        "NewTextFeature_173_word2vec(top_8)_173",
        "NewTextFeature_174_word2vec(top_8)_174",
        "NewTextFeature_175_word2vec(top_8)_175",
        "NewTextFeature_176_word2vec(top_8)_176",
        "NewTextFeature_177_word2vec(top_8)_177",
        "NewTextFeature_178_word2vec(top_8)_178",
        "NewTextFeature_179_word2vec(top_8)_179",
    ],
    missing_values_reference_list=["?", "", "-", float("nan")],
    options=["all"],
)
numpy_column_selector_1 = NumpyColumnSelector(
    columns=[
        0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
        71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
        88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
        132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,
        146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
        160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
        174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 187,
    ]
)
float_str2_float = FloatStr2Float(
    dtypes_list=[
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_num", "float_num",
        "float_num", "float_num", "float_num", "float_int_num", "int_num",
    ],
    missing_values_reference_list=[],
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    filling_values=float("nan"), missing_values=[]
)
num_imputer = NumImputer(missing_values=float("nan"), strategy="median")
opt_standard_scaler = OptStandardScaler(use_scaler_flag=False)
pipeline_1 = make_pipeline(
    text_transformer_1,
    date_transformer_1,
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        184, 185, 186, 188, 189, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
        14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
        31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
        48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
        65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
        82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98,
        99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
        127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
        141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
        155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
        183, 187,
    ],
)
ta1 = TA1(
    fun=np.sqrt,
    name="sqrt",
    datatypes=["numeric"],
    feat_constraints=[
        autoai_libs.utils.fc_methods.is_non_negative,
        autoai_libs.utils.fc_methods.is_not_categorical,
    ],
    col_names=[
        "open", "high", "low", "NewTextFeature_0_word2vec(top_0)_0",
        "NewTextFeature_1_word2vec(top_0)_1",
        "NewTextFeature_2_word2vec(top_0)_2",
        "NewTextFeature_3_word2vec(top_0)_3",
        "NewTextFeature_4_word2vec(top_0)_4",
        "NewTextFeature_5_word2vec(top_0)_5",
        "NewTextFeature_6_word2vec(top_0)_6",
        "NewTextFeature_7_word2vec(top_0)_7",
        "NewTextFeature_8_word2vec(top_0)_8",
        "NewTextFeature_9_word2vec(top_0)_9",
        "NewTextFeature_10_word2vec(top_0)_10",
        "NewTextFeature_11_word2vec(top_0)_11",
        "NewTextFeature_12_word2vec(top_0)_12",
        "NewTextFeature_13_word2vec(top_0)_13",
        "NewTextFeature_14_word2vec(top_0)_14",
        "NewTextFeature_15_word2vec(top_0)_15",
        "NewTextFeature_16_word2vec(top_0)_16",
        "NewTextFeature_17_word2vec(top_0)_17",
        "NewTextFeature_18_word2vec(top_0)_18",
        "NewTextFeature_19_word2vec(top_0)_19",
        "NewTextFeature_20_word2vec(top_1)_20",
        "NewTextFeature_21_word2vec(top_1)_21",
        "NewTextFeature_22_word2vec(top_1)_22",
        "NewTextFeature_23_word2vec(top_1)_23",
        "NewTextFeature_24_word2vec(top_1)_24",
        "NewTextFeature_25_word2vec(top_1)_25",
        "NewTextFeature_26_word2vec(top_1)_26",
        "NewTextFeature_27_word2vec(top_1)_27",
        "NewTextFeature_28_word2vec(top_1)_28",
        "NewTextFeature_29_word2vec(top_1)_29",
        "NewTextFeature_30_word2vec(top_1)_30",
        "NewTextFeature_31_word2vec(top_1)_31",
        "NewTextFeature_32_word2vec(top_1)_32",
        "NewTextFeature_33_word2vec(top_1)_33",
        "NewTextFeature_34_word2vec(top_1)_34",
        "NewTextFeature_35_word2vec(top_1)_35",
        "NewTextFeature_36_word2vec(top_1)_36",
        "NewTextFeature_37_word2vec(top_1)_37",
        "NewTextFeature_38_word2vec(top_1)_38",
        "NewTextFeature_39_word2vec(top_1)_39",
        "NewTextFeature_40_word2vec(top_2)_40",
        "NewTextFeature_41_word2vec(top_2)_41",
        "NewTextFeature_42_word2vec(top_2)_42",
        "NewTextFeature_43_word2vec(top_2)_43",
        "NewTextFeature_44_word2vec(top_2)_44",
        "NewTextFeature_45_word2vec(top_2)_45",
        "NewTextFeature_46_word2vec(top_2)_46",
        "NewTextFeature_47_word2vec(top_2)_47",
        "NewTextFeature_48_word2vec(top_2)_48",
        "NewTextFeature_49_word2vec(top_2)_49",
        "NewTextFeature_50_word2vec(top_2)_50",
        "NewTextFeature_51_word2vec(top_2)_51",
        "NewTextFeature_52_word2vec(top_2)_52",
        "NewTextFeature_53_word2vec(top_2)_53",
        "NewTextFeature_54_word2vec(top_2)_54",
        "NewTextFeature_55_word2vec(top_2)_55",
        "NewTextFeature_56_word2vec(top_2)_56",
        "NewTextFeature_57_word2vec(top_2)_57",
        "NewTextFeature_58_word2vec(top_2)_58",
        "NewTextFeature_59_word2vec(top_2)_59",
        "NewTextFeature_60_word2vec(top_3)_60",
        "NewTextFeature_61_word2vec(top_3)_61",
        "NewTextFeature_62_word2vec(top_3)_62",
        "NewTextFeature_63_word2vec(top_3)_63",
        "NewTextFeature_64_word2vec(top_3)_64",
        "NewTextFeature_65_word2vec(top_3)_65",
        "NewTextFeature_66_word2vec(top_3)_66",
        "NewTextFeature_67_word2vec(top_3)_67",
        "NewTextFeature_68_word2vec(top_3)_68",
        "NewTextFeature_69_word2vec(top_3)_69",
        "NewTextFeature_70_word2vec(top_3)_70",
        "NewTextFeature_71_word2vec(top_3)_71",
        "NewTextFeature_72_word2vec(top_3)_72",
        "NewTextFeature_73_word2vec(top_3)_73",
        "NewTextFeature_74_word2vec(top_3)_74",
        "NewTextFeature_75_word2vec(top_3)_75",
        "NewTextFeature_76_word2vec(top_3)_76",
        "NewTextFeature_77_word2vec(top_3)_77",
        "NewTextFeature_78_word2vec(top_3)_78",
        "NewTextFeature_79_word2vec(top_3)_79",
        "NewTextFeature_80_word2vec(top_4)_80",
        "NewTextFeature_81_word2vec(top_4)_81",
        "NewTextFeature_82_word2vec(top_4)_82",
        "NewTextFeature_83_word2vec(top_4)_83",
        "NewTextFeature_84_word2vec(top_4)_84",
        "NewTextFeature_85_word2vec(top_4)_85",
        "NewTextFeature_86_word2vec(top_4)_86",
        "NewTextFeature_87_word2vec(top_4)_87",
        "NewTextFeature_88_word2vec(top_4)_88",
        "NewTextFeature_89_word2vec(top_4)_89",
        "NewTextFeature_90_word2vec(top_4)_90",
        "NewTextFeature_91_word2vec(top_4)_91",
        "NewTextFeature_92_word2vec(top_4)_92",
        "NewTextFeature_93_word2vec(top_4)_93",
        "NewTextFeature_94_word2vec(top_4)_94",
        "NewTextFeature_95_word2vec(top_4)_95",
        "NewTextFeature_96_word2vec(top_4)_96",
        "NewTextFeature_97_word2vec(top_4)_97",
        "NewTextFeature_98_word2vec(top_4)_98",
        "NewTextFeature_99_word2vec(top_4)_99",
        "NewTextFeature_100_word2vec(top_5)_100",
        "NewTextFeature_101_word2vec(top_5)_101",
        "NewTextFeature_102_word2vec(top_5)_102",
        "NewTextFeature_103_word2vec(top_5)_103",
        "NewTextFeature_104_word2vec(top_5)_104",
        "NewTextFeature_105_word2vec(top_5)_105",
        "NewTextFeature_106_word2vec(top_5)_106",
        "NewTextFeature_107_word2vec(top_5)_107",
        "NewTextFeature_108_word2vec(top_5)_108",
        "NewTextFeature_109_word2vec(top_5)_109",
        "NewTextFeature_110_word2vec(top_5)_110",
        "NewTextFeature_111_word2vec(top_5)_111",
        "NewTextFeature_112_word2vec(top_5)_112",
        "NewTextFeature_113_word2vec(top_5)_113",
        "NewTextFeature_114_word2vec(top_5)_114",
        "NewTextFeature_115_word2vec(top_5)_115",
        "NewTextFeature_116_word2vec(top_5)_116",
        "NewTextFeature_117_word2vec(top_5)_117",
        "NewTextFeature_118_word2vec(top_5)_118",
        "NewTextFeature_119_word2vec(top_5)_119",
        "NewTextFeature_120_word2vec(top_6)_120",
        "NewTextFeature_121_word2vec(top_6)_121",
        "NewTextFeature_122_word2vec(top_6)_122",
        "NewTextFeature_123_word2vec(top_6)_123",
        "NewTextFeature_124_word2vec(top_6)_124",
        "NewTextFeature_125_word2vec(top_6)_125",
        "NewTextFeature_126_word2vec(top_6)_126",
        "NewTextFeature_127_word2vec(top_6)_127",
        "NewTextFeature_128_word2vec(top_6)_128",
        "NewTextFeature_129_word2vec(top_6)_129",
        "NewTextFeature_130_word2vec(top_6)_130",
        "NewTextFeature_131_word2vec(top_6)_131",
        "NewTextFeature_132_word2vec(top_6)_132",
        "NewTextFeature_133_word2vec(top_6)_133",
        "NewTextFeature_134_word2vec(top_6)_134",
        "NewTextFeature_135_word2vec(top_6)_135",
        "NewTextFeature_136_word2vec(top_6)_136",
        "NewTextFeature_137_word2vec(top_6)_137",
        "NewTextFeature_138_word2vec(top_6)_138",
        "NewTextFeature_139_word2vec(top_6)_139",
        "NewTextFeature_140_word2vec(top_7)_140",
        "NewTextFeature_141_word2vec(top_7)_141",
        "NewTextFeature_142_word2vec(top_7)_142",
        "NewTextFeature_143_word2vec(top_7)_143",
        "NewTextFeature_144_word2vec(top_7)_144",
        "NewTextFeature_145_word2vec(top_7)_145",
        "NewTextFeature_146_word2vec(top_7)_146",
        "NewTextFeature_147_word2vec(top_7)_147",
        "NewTextFeature_148_word2vec(top_7)_148",
        "NewTextFeature_149_word2vec(top_7)_149",
        "NewTextFeature_150_word2vec(top_7)_150",
        "NewTextFeature_151_word2vec(top_7)_151",
        "NewTextFeature_152_word2vec(top_7)_152",
        "NewTextFeature_153_word2vec(top_7)_153",
        "NewTextFeature_154_word2vec(top_7)_154",
        "NewTextFeature_155_word2vec(top_7)_155",
        "NewTextFeature_156_word2vec(top_7)_156",
        "NewTextFeature_157_word2vec(top_7)_157",
        "NewTextFeature_158_word2vec(top_7)_158",
        "NewTextFeature_159_word2vec(top_7)_159",
        "NewTextFeature_160_word2vec(top_8)_160",
        "NewTextFeature_161_word2vec(top_8)_161",
        "NewTextFeature_162_word2vec(top_8)_162",
        "NewTextFeature_163_word2vec(top_8)_163",
        "NewTextFeature_164_word2vec(top_8)_164",
        "NewTextFeature_165_word2vec(top_8)_165",
        "NewTextFeature_166_word2vec(top_8)_166",
        "NewTextFeature_167_word2vec(top_8)_167",
        "NewTextFeature_168_word2vec(top_8)_168",
        "NewTextFeature_169_word2vec(top_8)_169",
        "NewTextFeature_170_word2vec(top_8)_170",
        "NewTextFeature_171_word2vec(top_8)_171",
        "NewTextFeature_172_word2vec(top_8)_172",
        "NewTextFeature_173_word2vec(top_8)_173",
        "NewTextFeature_174_word2vec(top_8)_174",
        "NewTextFeature_175_word2vec(top_8)_175",
        "NewTextFeature_176_word2vec(top_8)_176",
        "NewTextFeature_177_word2vec(top_8)_177",
        "NewTextFeature_178_word2vec(top_8)_178",
        "NewTextFeature_179_word2vec(top_8)_179",
        "NewDateFeature_0_FloatTimestamp(date)",
        "NewDateFeature_1_Year(date)", "NewDateFeature_2_Month(date)",
        "NewDateFeature_3_Week(date)", "NewDateFeature_4_DayOfYear(date)",
        "NewDateFeature_5_DayOfMonth(date)",
        "NewDateFeature_6_DayOfWeek(date)",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"),
    ],
)
fs1_0 = FS1(
    cols_ids_must_keep=range(0, 190),
    additional_col_count_to_keep=36,
    ptype="regression",
)
tam = TAM(
    tans_class=PCA(),
    name="pca",
    col_names=[
        "open", "high", "low", "NewTextFeature_0_word2vec(top_0)_0",
        "NewTextFeature_1_word2vec(top_0)_1",
        "NewTextFeature_2_word2vec(top_0)_2",
        "NewTextFeature_3_word2vec(top_0)_3",
        "NewTextFeature_4_word2vec(top_0)_4",
        "NewTextFeature_5_word2vec(top_0)_5",
        "NewTextFeature_6_word2vec(top_0)_6",
        "NewTextFeature_7_word2vec(top_0)_7",
        "NewTextFeature_8_word2vec(top_0)_8",
        "NewTextFeature_9_word2vec(top_0)_9",
        "NewTextFeature_10_word2vec(top_0)_10",
        "NewTextFeature_11_word2vec(top_0)_11",
        "NewTextFeature_12_word2vec(top_0)_12",
        "NewTextFeature_13_word2vec(top_0)_13",
        "NewTextFeature_14_word2vec(top_0)_14",
        "NewTextFeature_15_word2vec(top_0)_15",
        "NewTextFeature_16_word2vec(top_0)_16",
        "NewTextFeature_17_word2vec(top_0)_17",
        "NewTextFeature_18_word2vec(top_0)_18",
        "NewTextFeature_19_word2vec(top_0)_19",
        "NewTextFeature_20_word2vec(top_1)_20",
        "NewTextFeature_21_word2vec(top_1)_21",
        "NewTextFeature_22_word2vec(top_1)_22",
        "NewTextFeature_23_word2vec(top_1)_23",
        "NewTextFeature_24_word2vec(top_1)_24",
        "NewTextFeature_25_word2vec(top_1)_25",
        "NewTextFeature_26_word2vec(top_1)_26",
        "NewTextFeature_27_word2vec(top_1)_27",
        "NewTextFeature_28_word2vec(top_1)_28",
        "NewTextFeature_29_word2vec(top_1)_29",
        "NewTextFeature_30_word2vec(top_1)_30",
        "NewTextFeature_31_word2vec(top_1)_31",
        "NewTextFeature_32_word2vec(top_1)_32",
        "NewTextFeature_33_word2vec(top_1)_33",
        "NewTextFeature_34_word2vec(top_1)_34",
        "NewTextFeature_35_word2vec(top_1)_35",
        "NewTextFeature_36_word2vec(top_1)_36",
        "NewTextFeature_37_word2vec(top_1)_37",
        "NewTextFeature_38_word2vec(top_1)_38",
        "NewTextFeature_39_word2vec(top_1)_39",
        "NewTextFeature_40_word2vec(top_2)_40",
        "NewTextFeature_41_word2vec(top_2)_41",
        "NewTextFeature_42_word2vec(top_2)_42",
        "NewTextFeature_43_word2vec(top_2)_43",
        "NewTextFeature_44_word2vec(top_2)_44",
        "NewTextFeature_45_word2vec(top_2)_45",
        "NewTextFeature_46_word2vec(top_2)_46",
        "NewTextFeature_47_word2vec(top_2)_47",
        "NewTextFeature_48_word2vec(top_2)_48",
        "NewTextFeature_49_word2vec(top_2)_49",
        "NewTextFeature_50_word2vec(top_2)_50",
        "NewTextFeature_51_word2vec(top_2)_51",
        "NewTextFeature_52_word2vec(top_2)_52",
        "NewTextFeature_53_word2vec(top_2)_53",
        "NewTextFeature_54_word2vec(top_2)_54",
        "NewTextFeature_55_word2vec(top_2)_55",
        "NewTextFeature_56_word2vec(top_2)_56",
        "NewTextFeature_57_word2vec(top_2)_57",
        "NewTextFeature_58_word2vec(top_2)_58",
        "NewTextFeature_59_word2vec(top_2)_59",
        "NewTextFeature_60_word2vec(top_3)_60",
        "NewTextFeature_61_word2vec(top_3)_61",
        "NewTextFeature_62_word2vec(top_3)_62",
        "NewTextFeature_63_word2vec(top_3)_63",
        "NewTextFeature_64_word2vec(top_3)_64",
        "NewTextFeature_65_word2vec(top_3)_65",
        "NewTextFeature_66_word2vec(top_3)_66",
        "NewTextFeature_67_word2vec(top_3)_67",
        "NewTextFeature_68_word2vec(top_3)_68",
        "NewTextFeature_69_word2vec(top_3)_69",
        "NewTextFeature_70_word2vec(top_3)_70",
        "NewTextFeature_71_word2vec(top_3)_71",
        "NewTextFeature_72_word2vec(top_3)_72",
        "NewTextFeature_73_word2vec(top_3)_73",
        "NewTextFeature_74_word2vec(top_3)_74",
        "NewTextFeature_75_word2vec(top_3)_75",
        "NewTextFeature_76_word2vec(top_3)_76",
        "NewTextFeature_77_word2vec(top_3)_77",
        "NewTextFeature_78_word2vec(top_3)_78",
        "NewTextFeature_79_word2vec(top_3)_79",
        "NewTextFeature_80_word2vec(top_4)_80",
        "NewTextFeature_81_word2vec(top_4)_81",
        "NewTextFeature_82_word2vec(top_4)_82",
        "NewTextFeature_83_word2vec(top_4)_83",
        "NewTextFeature_84_word2vec(top_4)_84",
        "NewTextFeature_85_word2vec(top_4)_85",
        "NewTextFeature_86_word2vec(top_4)_86",
        "NewTextFeature_87_word2vec(top_4)_87",
        "NewTextFeature_88_word2vec(top_4)_88",
        "NewTextFeature_89_word2vec(top_4)_89",
        "NewTextFeature_90_word2vec(top_4)_90",
        "NewTextFeature_91_word2vec(top_4)_91",
        "NewTextFeature_92_word2vec(top_4)_92",
        "NewTextFeature_93_word2vec(top_4)_93",
        "NewTextFeature_94_word2vec(top_4)_94",
        "NewTextFeature_95_word2vec(top_4)_95",
        "NewTextFeature_96_word2vec(top_4)_96",
        "NewTextFeature_97_word2vec(top_4)_97",
        "NewTextFeature_98_word2vec(top_4)_98",
        "NewTextFeature_99_word2vec(top_4)_99",
        "NewTextFeature_100_word2vec(top_5)_100",
        "NewTextFeature_101_word2vec(top_5)_101",
        "NewTextFeature_102_word2vec(top_5)_102",
        "NewTextFeature_103_word2vec(top_5)_103",
        "NewTextFeature_104_word2vec(top_5)_104",
        "NewTextFeature_105_word2vec(top_5)_105",
        "NewTextFeature_106_word2vec(top_5)_106",
        "NewTextFeature_107_word2vec(top_5)_107",
        "NewTextFeature_108_word2vec(top_5)_108",
        "NewTextFeature_109_word2vec(top_5)_109",
        "NewTextFeature_110_word2vec(top_5)_110",
        "NewTextFeature_111_word2vec(top_5)_111",
        "NewTextFeature_112_word2vec(top_5)_112",
        "NewTextFeature_113_word2vec(top_5)_113",
        "NewTextFeature_114_word2vec(top_5)_114",
        "NewTextFeature_115_word2vec(top_5)_115",
        "NewTextFeature_116_word2vec(top_5)_116",
        "NewTextFeature_117_word2vec(top_5)_117",
        "NewTextFeature_118_word2vec(top_5)_118",
        "NewTextFeature_119_word2vec(top_5)_119",
        "NewTextFeature_120_word2vec(top_6)_120",
        "NewTextFeature_121_word2vec(top_6)_121",
        "NewTextFeature_122_word2vec(top_6)_122",
        "NewTextFeature_123_word2vec(top_6)_123",
        "NewTextFeature_124_word2vec(top_6)_124",
        "NewTextFeature_125_word2vec(top_6)_125",
        "NewTextFeature_126_word2vec(top_6)_126",
        "NewTextFeature_127_word2vec(top_6)_127",
        "NewTextFeature_128_word2vec(top_6)_128",
        "NewTextFeature_129_word2vec(top_6)_129",
        "NewTextFeature_130_word2vec(top_6)_130",
        "NewTextFeature_131_word2vec(top_6)_131",
        "NewTextFeature_132_word2vec(top_6)_132",
        "NewTextFeature_133_word2vec(top_6)_133",
        "NewTextFeature_134_word2vec(top_6)_134",
        "NewTextFeature_135_word2vec(top_6)_135",
        "NewTextFeature_136_word2vec(top_6)_136",
        "NewTextFeature_137_word2vec(top_6)_137",
        "NewTextFeature_138_word2vec(top_6)_138",
        "NewTextFeature_139_word2vec(top_6)_139",
        "NewTextFeature_140_word2vec(top_7)_140",
        "NewTextFeature_141_word2vec(top_7)_141",
        "NewTextFeature_142_word2vec(top_7)_142",
        "NewTextFeature_143_word2vec(top_7)_143",
        "NewTextFeature_144_word2vec(top_7)_144",
        "NewTextFeature_145_word2vec(top_7)_145",
        "NewTextFeature_146_word2vec(top_7)_146",
        "NewTextFeature_147_word2vec(top_7)_147",
        "NewTextFeature_148_word2vec(top_7)_148",
        "NewTextFeature_149_word2vec(top_7)_149",
        "NewTextFeature_150_word2vec(top_7)_150",
        "NewTextFeature_151_word2vec(top_7)_151",
        "NewTextFeature_152_word2vec(top_7)_152",
        "NewTextFeature_153_word2vec(top_7)_153",
        "NewTextFeature_154_word2vec(top_7)_154",
        "NewTextFeature_155_word2vec(top_7)_155",
        "NewTextFeature_156_word2vec(top_7)_156",
        "NewTextFeature_157_word2vec(top_7)_157",
        "NewTextFeature_158_word2vec(top_7)_158",
        "NewTextFeature_159_word2vec(top_7)_159",
        "NewTextFeature_160_word2vec(top_8)_160",
        "NewTextFeature_161_word2vec(top_8)_161",
        "NewTextFeature_162_word2vec(top_8)_162",
        "NewTextFeature_163_word2vec(top_8)_163",
        "NewTextFeature_164_word2vec(top_8)_164",
        "NewTextFeature_165_word2vec(top_8)_165",
        "NewTextFeature_166_word2vec(top_8)_166",
        "NewTextFeature_167_word2vec(top_8)_167",
        "NewTextFeature_168_word2vec(top_8)_168",
        "NewTextFeature_169_word2vec(top_8)_169",
        "NewTextFeature_170_word2vec(top_8)_170",
        "NewTextFeature_171_word2vec(top_8)_171",
        "NewTextFeature_172_word2vec(top_8)_172",
        "NewTextFeature_173_word2vec(top_8)_173",
        "NewTextFeature_174_word2vec(top_8)_174",
        "NewTextFeature_175_word2vec(top_8)_175",
        "NewTextFeature_176_word2vec(top_8)_176",
        "NewTextFeature_177_word2vec(top_8)_177",
        "NewTextFeature_178_word2vec(top_8)_178",
        "NewTextFeature_179_word2vec(top_8)_179",
        "NewDateFeature_0_FloatTimestamp(date)",
        "NewDateFeature_1_Year(date)", "NewDateFeature_2_Month(date)",
        "NewDateFeature_3_Week(date)", "NewDateFeature_4_DayOfYear(date)",
        "NewDateFeature_5_DayOfMonth(date)",
        "NewDateFeature_6_DayOfWeek(date)", "sqrt(open)", "sqrt(high)",
        "sqrt(low)", "sqrt(NewTextFeature_60_word2vec(top_3)_60)",
        "sqrt(NewTextFeature_65_word2vec(top_3)_65)",
        "sqrt(NewTextFeature_80_word2vec(top_4)_80)",
        "sqrt(NewTextFeature_84_word2vec(top_4)_84)",
        "sqrt(NewTextFeature_85_word2vec(top_4)_85)",
        "sqrt(NewTextFeature_93_word2vec(top_4)_93)",
        "sqrt(NewTextFeature_95_word2vec(top_4)_95)",
        "sqrt(NewTextFeature_97_word2vec(top_4)_97)",
        "sqrt(NewTextFeature_100_word2vec(top_5)_100)",
        "sqrt(NewTextFeature_102_word2vec(top_5)_102)",
        "sqrt(NewTextFeature_103_word2vec(top_5)_103)",
        "sqrt(NewTextFeature_108_word2vec(top_5)_108)",
        "sqrt(NewTextFeature_110_word2vec(top_5)_110)",
        "sqrt(NewTextFeature_111_word2vec(top_5)_111)",
        "sqrt(NewTextFeature_113_word2vec(top_5)_113)",
        "sqrt(NewTextFeature_115_word2vec(top_5)_115)",
        "sqrt(NewTextFeature_118_word2vec(top_5)_118)",
        "sqrt(NewTextFeature_119_word2vec(top_5)_119)",
        "sqrt(NewTextFeature_120_word2vec(top_6)_120)",
        "sqrt(NewTextFeature_122_word2vec(top_6)_122)",
        "sqrt(NewTextFeature_123_word2vec(top_6)_123)",
        "sqrt(NewTextFeature_125_word2vec(top_6)_125)",
        "sqrt(NewTextFeature_127_word2vec(top_6)_127)",
        "sqrt(NewTextFeature_129_word2vec(top_6)_129)",
        "sqrt(NewTextFeature_134_word2vec(top_6)_134)",
        "sqrt(NewTextFeature_135_word2vec(top_6)_135)",
        "sqrt(NewTextFeature_137_word2vec(top_6)_137)",
        "sqrt(NewTextFeature_140_word2vec(top_7)_140)",
        "sqrt(NewTextFeature_145_word2vec(top_7)_145)",
        "sqrt(NewTextFeature_157_word2vec(top_7)_157)",
        "sqrt(NewTextFeature_160_word2vec(top_8)_160)",
        "sqrt(NewTextFeature_165_word2vec(top_8)_165)",
        "sqrt(NewDateFeature_0_FloatTimestamp(date))",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"),
    ],
)
fs1_1 = FS1(
    cols_ids_must_keep=range(0, 190),
    additional_col_count_to_keep=36,
    ptype="regression",
)
linear_regression = LinearRegression(n_jobs=CPU_NUMBER)


#### Pipeline.

In [ ]:
pipeline = make_pipeline(
    union, numpy_permute_array, ta1, fs1_0, tam, fs1_1, linear_regression
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [ ]:
from sklearn.metrics import make_scorer

from autoai_libs.scorers.scorers import neg_root_mean_squared_error

scorer = make_scorer(neg_root_mean_squared_error)

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [ ]:
pipeline.fit(X_train.values, y_train.values.ravel());

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [ ]:
score = scorer(pipeline, X_test.values, y_test.values)
print(score)

In [ ]:
pipeline.predict(X_test.values[:5])

<a id="saving"></a>
## Store the model

In this section you will learn how to store the trained model.

In [ ]:
model_metadata = {
    wml_client.repository.ModelMetaNames.NAME: 'P4 - Pretrained AutoAI pipeline'
}

stored_model_details = wml_client.repository.store_model(model=pipeline, meta_props=model_metadata, experiment_metadata=experiment_metadata)

Inspect the stored model details.

In [ ]:
stored_model_details

<a id="deployment"></a>
## Create online deployment

You can use commands bellow to promote the model to space and create online deployment (web service).

<a id="working_spaces"></a>
### Working with spaces

In this section you will specify a deployment space for organizing the assets for deploying and scoring the model. If you do not have an existing space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create a new space, following these steps:

- Click **New Deployment Space**.
- Create an empty space.
- Select Cloud Object Storage.
- Select Watson Machine Learning instance and press **Create**.
- Copy `space_id` and paste it below.

**Tip**: You can also use the API to prepare the space for your work. Learn more [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign or update space ID below.

#### Prepare online deployment

#### Test online deployment

<a id="cleanup"></a>
### Deleting deployment
You can delete the existing deployment by calling the `wml_client.deployments.delete(deployment_id)` command.
To list the existing web services, use `wml_client.deployments.list()`.

<a id="summary_and_next_steps"></a>
# Summary and next steps
You successfully completed this notebook!
You learned how to use AutoAI pipeline definition to train the model.
Check out our [Online Documentation](https://www.ibm.com/cloud/watson-studio/autoai) for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>

___